In [ ]:
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Get hands-on with a customer support use case using Gemini and Gen AI SDK

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fgenerative-ai%2Fmain%2Fgemini%2Fuse-cases%2Fcustomer-support%2Fcustomer_support_gemini_genai_sdk.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/generative-ai/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>

<div style="clear: both;"></div>

<b>Share to:</b>

<a href="https://www.linkedin.com/sharing/share-offsite/?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/8/81/LinkedIn_icon.svg" alt="LinkedIn logo">
</a>

<a href="https://bsky.app/intent/compose?text=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/7/7a/Bluesky_Logo.svg" alt="Bluesky logo">
</a>

<a href="https://twitter.com/intent/tweet?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/5a/X_icon_2.svg" alt="X logo">
</a>

<a href="https://reddit.com/submit?url=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb" target="_blank">
  <img width="20px" src="https://redditinc.com/hubfs/Reddit%20Inc/Brand/Reddit_Logo.png" alt="Reddit logo">
</a>

<a href="https://www.facebook.com/sharer/sharer.php?u=https%3A//github.com/GoogleCloudPlatform/generative-ai/blob/main/gemini/use-cases/customer-support/customer_support_gemini_genai_sdk.ipynb" target="_blank">
  <img width="20px" src="https://upload.wikimedia.org/wikipedia/commons/5/51/Facebook_f_logo_%282019%29.svg" alt="Facebook logo">
</a>

| Author |
| --- |
| [Eric Dong](https://github.com/gericdong) |

## Overview

This tutorial provides you hands-on experience building your own Gemini-powered applications. It guides you through practical examples and code snippets, focusing on a customer support use case, to show you how to start creating AI applications using Gemini and Google Gen AI SDK.

## Getting Started

### ⚙️ **Setup**: Install Gen AI SDK


In [ ]:
%pip install --upgrade --quiet google-genai

#### Authenticate your notebook environment

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

#### Set Google Cloud project and location

In [ ]:
import os

PROJECT_ID = "[your-project-id]"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = os.environ.get("GOOGLE_CLOUD_REGION", "us-central1")

#### Create a product catalog

In [ ]:
from google.cloud import storage

storage_client = storage.Client()
bucket = storage_client.bucket("cloud-samples-data")
blobs = bucket.list_blobs(prefix="generative-ai/image/retail")

id = 0
product_catalog = []
for blob in blobs:
    if blob.name.endswith(".png"):
        item = {
            "id": id,
            "inventory": "In stock",
            "price": 39.99,
            "image_url": blob.public_url,
        }
        product_catalog.append(item)
        id += 1

### 1️⃣ Import libraries


In [ ]:
from IPython.display import HTML, Audio, Image, Markdown, display
from google import genai
from google.genai.types import (
    Content,
    FunctionDeclaration,
    GenerateContentConfig,
    GoogleSearch,
    LiveConnectConfig,
    Part,
    Tool,
)
import numpy as np

### 2️⃣ Create a client

In [ ]:
client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)

## Customer support use case

Imagine a customer is buying a chair. He searched the internet and got a picture of a chair that he likes from the web, then he is reaching out online to his favorite furniture store for assistance.


#### 🤔 **Example question 1**: Do you have a chair similar to the one in this picture, but in red?


In [ ]:
customer_query = "Do you have chairs similar to the one in this picture, but in red?"

In [ ]:
customer_chair_image_url = (
    "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/armchair.png"
)

display(Image(url=customer_chair_image_url, width=300))

### 3️⃣ Send requests to the model to generate content

The request may contain:
- `model`: The model ID
- `contents`: Multimodal input (text, PDFs, images, audio, video)
- `config`: Configuration for generation

Learn more about [Google models](https://cloud.google.com/vertex-ai/generative-ai/docs/learn/models).

In [ ]:
MODEL_ID = "gemini-2.5-pro"  # gemini-2.5-pro, gemini-2.0-flash, gemini-2.0-flash-lite

Use `contents` to construct multimodal input including text, PDFs, images, audio, video, and code all together.

Here, you can leverage the long context window that helps seamlessly analyze large amounts of information.

In [ ]:
product_catalog_parts = []
for product in product_catalog:
    product_catalog_parts.append(f"Chair (id={product['id']}):")
    product_catalog_parts.append(
        Part.from_uri(file_uri=product["image_url"], mime_type="image/png")
    )


contents = [
    customer_query,
    Part.from_uri(file_uri=customer_chair_image_url, mime_type="image/png"),
    "catalog:",
    product_catalog_parts,
]

Use `system_instruction` to give the model additional context to understand the task, provide more customized responses, adhere to specific guidelines over the full user interaction with the model.

Gemini 2.0 models were trained to have conversational responses that are short, to the point and cost-effective for serving. If you need richer, more verbose answers use the following system instruction:


> All questions should be answered comprehensively with details,
unless the user requests a concise response specifically.


In [ ]:
system_instruction = """
You are an expert sales assistant specializing in furniture recommendations.
"""

Use `client.models.generate_content` to send a request to the model with `model`, `contents`, `config`, `tools`, etc.


In [ ]:
%%time

response = client.models.generate_content(
    model=MODEL_ID,
    contents=contents,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

### 4️⃣ Read model responses

The response may contain:
- Multimodal output (text, code, images, audio, embeddings)
- Response metadata

In [ ]:
response

You can use `Markdown` to display the formatted text.


In [ ]:
display(Markdown(response.text))

### 🚀 **Feature**: Structured output (controlled generation)

When you want the model response to a prompt to be returned in a structured data format, particularly if you are using the responses for downstream processes, such as downstream modules that expect a specific format as input, you use the controlled generation to ensure that model outputs comply with a specific structured format.

You define a response schema for the model output using these two options:

- **Option 1**: Using a Pydantic object

In [ ]:
from pydantic import BaseModel, RootModel


class MatchedFurnitureInfo(BaseModel):
    id: int
    match_score: int
    match_reason: str


class MatchedFurnitureList(RootModel):
    root: list[MatchedFurnitureInfo]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]

- **Option 2**: Using a dictionary as in OpenAPI schema.


In [ ]:
response_schema = {
    "type": "ARRAY",
    "items": {
        "type": "OBJECT",
        "properties": {
            "id": {"type": "INTEGER"},
            "match_reason": {"type": "STRING", "nullable": True},
        },
    },
}

This example uses the OpenAPI schema option, then sets the defined schema in the `response_schema` in the `GenerateContentConfig`.

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=response.text,
    config=GenerateContentConfig(
        system_instruction="Convert the given text to JSON",
        response_mime_type="application/json",
        response_schema=response_schema,
    ),
)

In [ ]:
response.parsed

In [ ]:
matching_items: dict = response.parsed

display(HTML("<style>#output-body{display:flex; flex-direction: row;}</style>"))
display(HTML("Original: "), Image(url=customer_chair_image_url, width=200))
display(HTML("Recommended: "))

matching_items_urls = []
for item in matching_items:
    url = product_catalog[item["id"]]["image_url"]
    display(Image(url=url, width=200))
    matching_items_urls.append(url)

### 🤔 **Example question 2**: Would the chair fit in my room?

- Reasoning across image and text

In [ ]:
customer_query = "Would the chair fit in my room?"

In [ ]:
customer_room_image_url = "https://storage.googleapis.com/cloud-samples-data/generative-ai/image/living-room-2.png"

display(Image(url=customer_room_image_url, width=400))

### Send request to the model

Here, you leverage the multimodal understanding to reason across the images of both the chair and the room.

In [ ]:
system_instruction = """
  You are an interior designer.
  Your mission is to help customers to create living spaces that balance functionality
  and beauty through personalized service and sustainable design practices.
"""

contents = [
    "Chair:",
    Part.from_uri(file_uri=matching_items_urls[0], mime_type="image/png"),
    "Living room:",
    Part.from_uri(file_uri=customer_room_image_url, mime_type="image/png"),
    customer_query,
]

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=contents,
    config=GenerateContentConfig(
        system_instruction=system_instruction,
    ),
)

display(Markdown(response.text))

### 🚀 **Feature**: Image generation

You also generate an image with the chair being placed in the living room, showcasing the Gemini's ability to generate images natively.

In [ ]:
response = client.models.generate_content(
    model="gemini-2.0-flash-preview-image-generation",
    contents=[
        contents,
        "Create an image with the chair be integrated in the living room",
    ],
    config=GenerateContentConfig(
        response_modalities=["TEXT", "IMAGE"],
    ),
)

for part in response.candidates[0].content.parts:
    if part.text:
        display(Markdown(part.text))
    if part.inline_data:
        display(Image(data=part.inline_data.data, width=400))

### 🤔 **Example question 3**: Is this chair available at a store near me? I am at Google Cloud Next!

Here you use the function calling and native tool use to enable models to connect to built-in or external tooling and systems, and fetch real-time or business data.


In [ ]:
customer_query = "Do you have the chair available in a store near me? I am at Google Cloud Next 2025."

### 🚀 **Feature**: Function calling

Define a `FunctionDeclaration` with the name of a function that matches the description and the arguments to call it with. You also can define a Python function for automatic function calling, which will run the function and return the output in natural language generated by the model.

In [ ]:
get_product_info_function = FunctionDeclaration(
    name="get_product_info",
    description="Get the stock amount and identifier for a given product",
    parameters={
        "type": "OBJECT",
        "properties": {
            "product_name": {"type": "STRING", "description": "Product name"}
        },
    },
)

get_store_location_function = FunctionDeclaration(
    name="get_store_location",
    description="Get the location of the closest store",
    parameters={
        "type": "OBJECT",
        "properties": {"location": {"type": "STRING", "description": "Location"}},
    },
)

Define a `Tool` that allows the model to select from a set of defined functions.

In [ ]:
retail_tool = Tool(
    function_declarations=[
        get_product_info_function,
        get_store_location_function,
    ],
)

Use function calling in a `chats` session to answer user's questions about the products.

In [ ]:
chat = client.chats.create(
    model="gemini-2.0-flash",
    config=GenerateContentConfig(
        temperature=0,
        tools=[retail_tool],
    ),
)

The model generates the function calls that you can use to connect to an external system to get real-time or business data.

In [ ]:
response = chat.send_message(customer_query)

response.function_calls

### 🚀 **Feature**: Google Search

Using `GoogleSearch` in Tool, Gemini will automatically do a Google search and incorporate information from the web into its responses. It also provides direct citations so you can easily fact-check sources.

In [ ]:
def get_store_location(location: str):
    google_search_tool = Tool(google_search=GoogleSearch())

    prompt = f"What is the location for {location}?"

    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=prompt,
        config=GenerateContentConfig(tools=[google_search_tool], temperature=0),
    )

    return {"store": response.text}


def get_product_info(product_name: str):
    return {"id": "3", "in_stock": "yes"}

This is where you use the model response to connect to external systems to get the real time or business data.

In [ ]:
function_response_parts = []

for function_call in response.function_calls:
    if function_call.name == "get_store_location":
        function_response = get_store_location(**function_call.args)
    elif function_call.name == "get_product_info":
        function_response = get_product_info(**function_call.args)
    else:
        raise ValueError(f"Unknown function: {function_call.name}")

    print(function_call.name)
    print(function_response)

    function_response_part = Part.from_function_response(
        name=function_call.name,
        response=function_response,
    )

    function_response_parts.append(function_response_part)

The mode incorporates the data from external systems, and returns the output in natural language.

In [ ]:
response = chat.send_message(function_response_parts)

display(Markdown(response.text))

### 🚀 **Feature**: Live API

#### **Example**: Text-to-audio conversation

**Step 1**: You set up a conversation with the API that allows you to send text prompts and receive audio responses.


In [ ]:
config = LiveConnectConfig(
    response_modalities=["AUDIO"],
    tools=[Tool(google_search=GoogleSearch())],
)


async def main() -> None:
    async with client.aio.live.connect(
        model="gemini-2.0-flash-live-preview-04-09", config=config
    ) as session:

        async def send() -> bool:
            text_input = input("Input > ")
            if text_input.lower() in ("q", "quit", "exit"):
                return False
            await session.send_client_content(
                turns=Content(role="user", parts=[Part(text=text_input)])
            )

            return True

        async def receive() -> None:

            audio_data = []

            async for message in session.receive():
                if (
                    message.server_content.model_turn
                    and message.server_content.model_turn.parts
                ):
                    for part in message.server_content.model_turn.parts:
                        if part.inline_data:
                            audio_data.append(
                                np.frombuffer(part.inline_data.data, dtype=np.int16)
                            )

                if message.server_content.turn_complete:
                    display(Markdown("**Response >**"))
                    display(
                        Audio(np.concatenate(audio_data), rate=24000, autoplay=True)
                    )
                    break

            return

        while True:
            if not await send():
                break
            await receive()

**Step 2** Run the chat, input your prompts, or type `q`, `quit` or `exit` to exit.

Sample questions:
- Where is the Google Cloud Next? Give me the address.
- How is the weather today?

In [ ]:
await main()

# 🎉 What's Next?

🚀 Learn more at the Google Cloud Gen AI sample repository 👇

[github.com/GoogleCloudPlatform/generative-ai](https://github.com/GoogleCloudPlatform/generative-ai)
